# AdaBoost vs Gradient Boosting Machine (GBM): Comparação e Otimização

## **1. Diferenças Fundamentais entre AdaBoost e GBM**

O **AdaBoost (Adaptive Boosting)** e o **Gradient Boosting Machine (GBM)** são ambos algoritmos de **ensemble learning**, mas possuem diferenças significativas em suas abordagens e funcionamento. A seguir, destacamos cinco diferenças principais:

### 🔹 **Método de Ajuste dos Erros**
**AdaBoost**: Ajusta os pesos das observações de acordo com os erros do modelo anterior. Os exemplos mal classificados recebem pesos maiores, tornando-os mais influentes na próxima iteração.
  
  $$ \alpha_t = \frac{1}{2} \ln \left( \frac{1 - e_t}{e_t} \right) $$

  onde:
  - $ e_t $ é a taxa de erro da árvore na iteração $ t $.
  - $ \alpha_t $ é o peso atribuído à árvore no ensemble.

**GBM**: Em vez de ajustar pesos, ele minimiza a **função de perda** ajustando cada novo modelo para prever o **gradiente do erro** do modelo anterior.

  $$ F_m(x) = F_{m-1}(x) + \gamma_m h_m(x) $$

  onde:
  - $ F_m(x) $ é o modelo atualizado na iteração $ m $.
  - $ h_m(x) $ é a nova árvore ajustada ao gradiente do erro.
  - $ \gamma_m $ é a taxa de aprendizado que controla a contribuição da nova árvore.

### 🔹 **Flexibilidade na Função de Perda**
**AdaBoost**: Usa **função de perda exponencial** por padrão:

  $$ L(y, F(x)) = e^{-yF(x)} $$

**GBM**: Permite a escolha de diversas funções de perda, incluindo:
  - **Erro quadrático** para regressão: $ L(y, \hat{y}) = (y - \hat{y})^2 $
  - **Log-loss** para classificação binária: $ L(y, \hat{y}) = - y \log(\hat{y}) - (1 - y) \log(1 - \hat{y}) $

### 🔹 **Sensibilidade a Outliers**
**AdaBoost**: Pode ser mais **sensível a outliers**, pois os exemplos difíceis de classificar recebem mais peso e podem influenciar excessivamente o modelo.

**GBM**: Possui mais **flexibilidade na função de perda**, permitindo que se adapte melhor a dados ruidosos.

### 🔹 **Complexidade Computacional**
**AdaBoost**: Normalmente **mais rápido**, pois apenas ajusta os pesos das observações a cada iteração.

**GBM**: **Mais lento**, pois precisa calcular o **gradiente do erro** e ajustar a árvore para minimizá-lo.

### 🔹 **Complexidade dos Modelos Base**
**AdaBoost**: Utiliza **stumps** (árvores rasas com profundidade 1) como base learners.

**GBM**: Pode usar árvores **mais profundas**, o que permite capturar relações mais complexas.

---

## **2. Implementação do GBM no Scikit-Learn**
Abaixo estão os exemplos de **classificação** e **regressão** utilizando o **Gradient Boosting**.

In [1]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, 
                                 learning_rate=1.0, 
                                 max_depth=1, 
                                 random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

### Exemplo 2: Regressão com GBM

In [3]:
import numpy as np

from sklearn.metrics  import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]

est = GradientBoostingRegressor(n_estimators=100, 
                                learning_rate=0.1, 
                                max_depth=1, 
                                random_state=0, 
                                loss='squared_error').fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))

5.009154859960319

## **3. Principais Hiperparâmetros do GBM**
A escolha dos hiperparâmetros é fundamental para o desempenho do modelo GBM. Aqui estão os mais importantes:

#### **`n_estimators` (Número de Árvores)**
- Define o **número de árvores** adicionadas ao ensemble.
- Quanto maior, mais preciso pode ser o modelo, mas aumenta o risco de **overfitting**.

#### **`learning_rate` (Taxa de Aprendizado)**
- Controla a **contribuição de cada árvore** no modelo final:

  $$ F_m(x) = F_{m-1}(x) + \gamma_m h_m(x) $$

- Taxas menores precisam de mais árvores para obter um bom desempenho.

#### **`max_depth` (Profundidade Máxima das Árvores)**
- Define a **complexidade das árvores**.
- **Valores altos** permitem capturar padrões complexos, mas podem causar **overfitting**.

#### **`max_features` (Número Máximo de Variáveis por Divisão)**
- Controla **quantas variáveis** são consideradas ao dividir cada nó.

#### **`warm_start` (Incrementação do Modelo)**
- Permite adicionar mais árvores ao modelo sem perder as anteriores.

---

## **4. Otimização de Hiperparâmetros com GridSearch**
Podemos usar **GridSearch** para encontrar os **melhores hiperparâmetros**.

In [5]:
import pandas as pd

In [6]:
%%time

estimators     = [10, 100, 1000, 10000]
learning_rates = [0.01, 0.03, 0.06, 0.1]
max_depths     = [1, 3, 6, 9]

grid_search = []

for n in estimators:
    for rate in learning_rates:
        for depth in max_depths:
            est = GradientBoostingRegressor(n_estimators=n, 
                                            learning_rate=rate, 
                                            max_depth=depth, 
                                            random_state=0, 
                                            loss='squared_error').fit(X_train, y_train)
            grid_search.append([n, rate, depth, mean_squared_error(y_test, est.predict(X_test))])
            
(pd.DataFrame(data=grid_search, 
              columns=['n_estimators', 'learning_rate', 'max_depth', 'mean_squared_error'])
   .sort_values(by='mean_squared_error', 
                ascending=True, 
                ignore_index=True))

CPU times: user 1min 25s, sys: 783 ms, total: 1min 25s
Wall time: 1min 29s


,n_estimators,learning_rate,max_depth,mean_squared_error
0,1000,0.03,1,3.625209
1,1000,0.06,1,3.667692
2,10000,0.10,3,3.733205
3,1000,0.10,3,3.733272
4,100,0.10,3,3.774987
...,...,...,...,...
59,10,0.03,1,21.909960
60,10,0.01,9,22.400111
61,10,0.01,6,22.472923
62,10,0.01,3,22.902975


## **5. Diferença entre GBM e Stochastic GBM**
O **Stochastic GBM** introduz **aleatoriedade** na seleção de amostras para cada árvore, reduzindo o **overfitting**.

- **GBM tradicional**: Usa **todo o conjunto de dados** em cada iteração.
- **Stochastic GBM**: Usa **subconjuntos aleatórios**, combinando **Bagging e Boosting**.

Isso melhora a **generalização do modelo** e reduz o tempo de treinamento.

---

### **Conclusão**
✅ **AdaBoost** é mais simples e rápido, mas sensível a outliers.  
✅ **GBM** é mais flexível, usa gradientes e suporta várias funções de perda.  
✅ **O ajuste fino dos hiperparâmetros é essencial** para maximizar o desempenho do GBM.  